In [1]:
!python -m spacy download nl_core_news_sm


[+] Download and installation successful
You can now load the package via spacy.load('nl_core_news_sm')


In [2]:
import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers
from spacy.tokens import DocBin
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import load_dataset
import nl_core_news_sm
import re
import numpy as np
torch.cuda.is_available(), spacy.prefer_gpu()


(True, True)

In [3]:
dataset = load_dataset("dutch_social")


Reusing dataset dutch_social (C:\Users\Emiel\.cache\huggingface\datasets\dutch_social\dutch_social\1.1.0\4ec8e931ab57f4a4483ad4b418676a45a7f6fec1cf6506da7d99c97259f7e02f)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])

In [5]:
train = train.sample(frac = 1)
test = test.sample(frac = 1)

In [6]:
train['label'] = train['label'].astype(str)
test['label'] = test['label'].astype(str)

In [7]:
def remove_link(text):
    return re.sub(r'https?:\/\/\S*', '', text, flags=re.MULTILINE)

def remove_username(text):
    return re.sub('@[\w]+','',text)

def remove_retweet(text):
    
    return text.replace("RT : ", "")

In [8]:
train['full_text'] = train['full_text'].apply(lambda x: remove_link(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_link(x))

train['full_text'] = train['full_text'].apply(lambda x: remove_username(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_username(x))

train['full_text'] = train['full_text'].apply(lambda x: remove_retweet(x))
test['full_text'] = test['full_text'].apply(lambda x: remove_retweet(x))

In [9]:
train_zip = list(zip(train['full_text'], train['label']))
test_zip = list(zip(test['full_text'], test['label']))


In [10]:
!pip install spacy
!pip install spacy-transformers

In [11]:
nlp=spacy.load("nl_core_news_sm")


In [12]:
def document(data):
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='0'):
      doc.cats['0'] = 1
      doc.cats['1'] = 0
      doc.cats['2']  = 0
    elif (label=='1'):
      doc.cats['0'] = 0
      doc.cats['1'] = 1
      doc.cats['2']  = 0
    elif (label=='2'):
      doc.cats['0'] = 0
      doc.cats['1'] = 0
      doc.cats['2']  = 1
    text.append(doc)
  
  return(text)

In [ ]:
train_docs = document(train_zip)
doc_bin = DocBin(docs = train_docs)
doc_bin.to_disk("train.spacy")

In [ ]:
test_docs = document(test_zip)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("valid.spacy")

In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg


In [ ]:
start_time = datetime.now()

!python -m spacy train config.cfg --verbose  --gpu-id 0 --output ./output_updated

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

In [ ]:
convert_dict = {0:'negatief', 1:'neutraal', 2:'positief'}

In [ ]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp("goed gedaan makker")
print(convert_dict[np.argmax(list(demo.cats.values()))])

In [ ]:
#Test the data from the best model
nlp = spacy.load("output_updated/model-best")
demo = nlp("dat heb je bijzonder slecht gedaan, idioot")
print(convert_dict[np.argmax(list(demo.cats.values()))])